# Project/Process
This notebook is a port of a repl.it with some new functions.

Notes and process:

*   Expanded and updated existing tweet-gathering functionality to sort the tweets by week number
*   DON'T FORGET to tell Ken that there's a transposition in the column names! (See 2:82)

Next steps:


*   Rewrite scraper/csv into class with relevant functions
*  Sort by week number + year to allow for multi-year processes
  * Finish implementing function that takes in different sorting params




In [ ]:
#! python3
!pip install openai

#libraries
import os
import openai

import datetime


     |████████████████████████████████| 152 kB 33.7 MB/s 
     |████████████████████████████████| 11.3 MB 44.2 MB/s 
     |████████████████████████████████| 161 kB 39.4 MB/s 
     |████████████████████████████████| 242 kB 50.9 MB/s 
  Created wheel for openai: filename=openai-0.11.4-py3-none-any.whl size=161960 sha256=f9f8566e4459a53104c539065263c5cc10db3781593fe171d53d9593dd6cf273
  Stored in directory: /root/.cache/pip/wheels/f6/60/b2/2275a37f6383cec638f9181c7289f2284ba574f8ddc7a836b8
Successfully built openai
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 2.5.9
    Uninstalling openpyxl-2.5.9:
      Successfully uninstalled openpyxl-2.5.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confli

In [ ]:
#! python3

#libraries
import pandas as pd
import tweepy
import csv
import os
import datetime

#Twitter API credentials
consumer_key = 'key'
consumer_secret = 'secret'
access_key = 'key'
access_secret = 'secret'


API_KEY = "api_key"



def validate_twitter(consumer_key, consumer_secret, access_key, access_secret):
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_key, access_secret)

  api = tweepy.API(auth)

  return api

def sort_tweets_by_time_chunk(input_df, column_name='Date', time_chunk="week"):
    for x in range(52):
      df = input_df[input_df[column_name].dt.isocalendar().week == x]
      print(df.count)
      # Only make folders and files for weeks where there were tweets
      if not df.empty:
        if not os.path.exists(f'week-{x}'):
          os.makedirs(f'week-{x}')
        df.to_csv(f'week-{x}/{screen_name}.csv')

# get an arbitrary # of tweets
def get_tweets(screen_name, num, api):
    new_tweets = api.user_timeline(screen_name=screen_name, count=num)
    
    return new_tweets


def get_all_tweets(screen_name, api):
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #initialize a list to hold all the tweepy Tweets
    alltweets = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name=screen_name, count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print(f"getting tweets before {oldest}")

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name=screen_name,
                                       count=200,
                                       max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print(f"...{len(alltweets)} tweets downloaded so far")

    #transform the tweepy tweets into a 2D array that will populate the csv
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text]
                 for tweet in alltweets]
                 
    # Transposed in the repl.it
    tweets_df = pd.DataFrame(outtweets,
                             columns=["Tweet_id", "Date", "Tweet_Text"])
    
    # TODO: incorporate this start to error handling
    # except BaseException as e:
    #       print('failed on_status', str(e))
    #       time.sleep(3)

  # For each week in the year, create dir if ! exists, filter df, write to csv, and save to folder
  # See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.weekofyear.html  
    for x in range(52):
      df = tweets_df[tweets_df['Date'].dt.isocalendar().week == x]
      print(df.count)
      # Only make folders and files for weeks where there were tweets
      if not df.empty:
        if not os.path.exists(f'week-{x}'):
          os.makedirs(f'week-{x}')
        df.to_csv(f'week-{x}/{screen_name}.csv')
    return tweets_df

    
# If using datetime object, can use this function to get the week for an individual tweet
# Note that every python data type handles this slightly differently! Check the type!
def make_chunks(data, tuple_position=1):
  # 0 = year, 1 = week, 2 = day
  weeks = []
  for x in data:
    chunk = x.created_at.isocalendar()[tuple_position]
    weeks.append(chunk)
  return weeks


#tester = get_all_tweets("Person_of_Note")
#print(tester)

# #tweets
# tweets = pd.read_csv('bored_tweets.csv')

# #remove links
# no_links = tweets[~tweets.text.str.contains("http")]
# no_links.head()

# # only tweets
# just_tweets = no_links.text

# # make everything a string
# text = str(just_tweets)


api = validate_twitter(consumer_key, consumer_secret, access_key, access_secret)
test = get_all_tweets('BoredElonMusk', api)



getting tweets before 1468316353207562240
...400 tweets downloaded so far
getting tweets before 1461113018259935235
...600 tweets downloaded so far
getting tweets before 1456270648209129479
...800 tweets downloaded so far
getting tweets before 1453446476915302401
...1000 tweets downloaded so far
getting tweets before 1445807030791331843
...1200 tweets downloaded so far
getting tweets before 1442989255974809600
...1400 tweets downloaded so far
getting tweets before 1439558651400781832
...1600 tweets downloaded so far
getting tweets before 1436051462065311745
...1800 tweets downloaded so far
getting tweets before 1433657589183631363
...2000 tweets downloaded so far
getting tweets before 1430385003783876612
...2200 tweets downloaded so far
getting tweets before 1427047982294126594
...2400 tweets downloaded so far
getting tweets before 1420738077388021760
...2600 tweets downloaded so far
getting tweets before 1411098812475592705
...2800 tweets downloaded so far
getting tweets before 140738

In [ ]:
# SUPPORT FUNCTIONS #
import sys
import shutil

# Lazy util function
def check_type(data):
  print(type(data))

def NUKE_ALL_FOLDERS_AND_FILES():
  print("Nuclear launch detected")
  # Get directory name
  # TODO: Update this to be more flexible, but it works for now
  for x in range(1,52):
    try:
        shutil.rmtree(f'week-{x}')
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))

#NUKE_ALL_FOLDERS_AND_FILES()

#os.listdir()

In [ ]:
#get api key
my_secret = "openai_key"
openai.api_key = my_secret

#get_all_tweets("BoredElonMusk")

In [ ]:
#build a new prompt
  #get a random sample of tweets to a list
rsample = corpus.completion.sample(10, random_state=1)
sample_list = rsample.tolist()
#write the prompt as a function string
prompt = f"Embody the mind of bored Elon Musk. Come up with more clever tweets like these:\n1.{sample_list[0]}\n2.{sample_list[1]}\n3.{sample_list[2]}\n4.{sample_list[3]}\n5.{sample_list[4]}\n6.{sample_list[5]}\n7.{sample_list[6]}\n8.{sample_list[7]}\n9.{sample_list[8]}\n10.{sample_list[9]}\n11.\n"
#rename and drop columns
corpus.rename(columns={"Tweet_Text":"completion"}, inplace=True)
corpus.drop(columns=["Date","Tweet_id"], inplace=True)

#drop @s and links
corpus = corpus[~corpus.completion.str.contains("@")]
corpus = corpus[~corpus.completion.str.contains("http")]

#add an empty prompt column
corpus["prompt"] = ""

#export the corpus to be used in fine-tuning
corpus.to_csv(f'filtered_tweets.csv', index=False)

In [ ]:
#define fine tuned model
FINE_TUNED_MODEL=os.environ["fine_tuned_model"]

#pick up where u left off with this youtube tutorial: https://www.youtube.com/watch?v=sIS_OgplWN8

def get_response(text):
  response = openai.Completion.create(
    model=FINE_TUNED_MODEL, #load it in
    #engine="ada", #engine
    prompt=text,  #prompt
    temperature=0.9, 
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop=["\n11.\n"]
  )
  return response.choices[0].text

summary = get_response(prompt)

print(summary)